In [1]:
import dataset_utils
import layers

c:\Users\maxde\anaconda3\envs\final311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\maxde\anaconda3\envs\final311\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
dataset = dataset_utils.SDFDataset("./cars100")

In [3]:
from torch.utils.data import DataLoader, random_split
import torch

# make dataloaders
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = layers.KLAutoEncoder().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [4]:
for a in train_loader:
    print(a.shape)
print(len(train_dataset))

torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
torch.Size([4, 50000, 4])
80


In [5]:
import trainer as t
# from importlib import reload
# reload(t)
t.train(model, train_loader, val_loader, optimizer, device, num_epochs=10)

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:48<00:00,  2.43s/it]


Epoch 1/10 - Validation Loss: 0.588572


100%|██████████| 20/20 [00:47<00:00,  2.37s/it]


Epoch 2/10 - Validation Loss: 0.145160


...

Epoch 8/10 - Validation Loss: 0.128479


100%|██████████| 20/20 [00:49<00:00,  2.46s/it]


Epoch 9/10 - Validation Loss: 0.135011


100%|██████████| 20/20 [00:49<00:00,  2.46s/it]


Epoch 10/10 - Validation Loss: 0.139078
